In [37]:
import numpy as np

with open("movie-xids.npy", "rb") as f:
    Xids = np.load(f, allow_pickle=True)
with open("movie-xmask.npy", "rb") as f:
    Xmask = np.load(f, allow_pickle=True)
with open("movie-labels.npy", "rb") as f:
    labels = np.load(f, allow_pickle=True)

In [38]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

<TakeDataset shapes: ((160,), (160,), (6,)), types: (tf.int32, tf.int32, tf.float64)>

In [39]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {"input_ids": input_ids,
            "attention_mask": masks}, labels

In [40]:
dataset = dataset.map(map_func)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (160,), attention_mask: (160,)}, (6,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [41]:
batch_size=16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 160), attention_mask: (16, 160)}, (16, 6)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [42]:
Xids[0], Xids[0].shape

(array([     2,  13158,  62480,  78805,   3671,  16421,   2767,   3100,
          5855, 106576, 120739,  67719,     47,   7647,   3156,   2416,
          1019,      3,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0, 

In [43]:
split = 0.9
size = int((Xids.shape[0] / batch_size) * split)
size

4681

In [44]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

len(dataset), len(train_ds), len(val_ds)

(5201, 4681, 520)

In [45]:
tf.data.experimental.save(train_ds, "train")
tf.data.experimental.save(val_ds, "val")

In [46]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 160), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 160), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 6), dtype=tf.float64, name=None))

In [47]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 160), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 160), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 6), dtype=tf.float64, name=None))

In [48]:
train_ds.element_spec == val_ds.element_spec

True